# Import

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import norm
import statsmodels.formula.api as smf
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from google.colab import drive
drive.mount('/content/drive')

# Data

In [ ]:
# Load data
path1 = "/content/drive/MyDrive/Thesis/Data/Merging CleanPrice & Features.csv"
price = pd.read_csv(path1, sep=',')

columns = price.copy()
price.set_index('Date', inplace=True)
RawData = price.copy()

# Making Mean Benchmark

In [ ]:
# Making rolling mean benchmark
columns = ['GEO_Name', 'Log_Return_h1', 'Log_Return_h3', 'Log_Return_h6', 'Log_Return_h12']
actual_data = RawData[columns].copy()
mean_dfs = []

for name, group in actual_data.groupby('GEO_Name'):
    group['Mean_h1'] = group['Log_Return_h1'].rolling(window=330).mean().shift(1)
    group['Mean_h3'] = group['Log_Return_h3'].rolling(window=330).mean().shift(3)
    group['Mean_h6'] = group['Log_Return_h6'].rolling(window=330).mean().shift(6)
    group['Mean_h12'] = group['Log_Return_h12'].rolling(window=330).mean().shift(12)
    # Append the results to df
    mean_dfs.append(group)

actual_data = pd.concat(mean_dfs).reset_index()
MeanBenchmark = actual_data[['Date', 'GEO_Name', 'Mean_h1', 'Mean_h3', 'Mean_h6', 'Mean_h12']].rename(columns={'GEO_Name': 'state', 'Mean_h1': 'h1', 'Mean_h3': 'h3', 'Mean_h6': 'h6', 'Mean_h12': 'h12'})

In [ ]:
# Loading predictions and corresponding actuals
actuals = pd.read_csv("/content/drive/MyDrive/Thesis/Models/Predictions/AR(1)Actuals.csv", sep=',')
AR_1 = pd.read_csv("/content/drive/MyDrive/Thesis/Models/Predictions/AR(1)Predictions.csv", sep=',')
AR_optimal = pd.read_csv("/content/drive/MyDrive/Thesis/Models/Predictions/AR(Optimal)Predictions.csv", sep=',')
ARIMA = pd.read_csv("/content/drive/MyDrive/Thesis/Models/Predictions/ARIMAPredictions.csv", sep=',')
RandomForest = pd.read_csv("/content/drive/MyDrive/Thesis/Models/Predictions/RandomForestPredictions.csv", sep=',')
XGBoost = pd.read_csv("/content/drive/MyDrive/Thesis/Models/Predictions/XGBoostPredictions.csv", sep=',')

In [ ]:
# Renaming the date column to date
actuals.rename(columns={'PredictionDate': 'Date'}, inplace=True)
AR_1.rename(columns={'PredictionDate': 'Date'}, inplace=True)
AR_optimal.rename(columns={'PredictionDate': 'Date'}, inplace=True)
ARIMA.rename(columns={'PredictionDate': 'Date'}, inplace=True)

RandomForest.rename(columns={'OriginalIndex': 'Date'}, inplace=True)
XGBoost.rename(columns={'OriginalIndex': 'Date'}, inplace=True)

RandomForest.rename(columns={'State': 'state'}, inplace=True)
XGBoost.rename(columns={'State': 'state'}, inplace=True)

In [ ]:
# Making Combination forecasts
combined_df1 = pd.concat([AR_optimal, ARIMA, RandomForest, XGBoost], ignore_index=True)
averaged_predictions1 = combined_df1.groupby(['Date', 'state']).mean().reset_index()
Combination_AR_1 = averaged_predictions1.sort_values(by=['state', 'Date']).reset_index(drop=True)

In [ ]:
# Removing 'USA' from the predictions and actuals
actuals = actuals[actuals['state'] != 'USA']
AR_1 = AR_1[AR_1['state'] != 'USA']
AR_optimal = AR_optimal[AR_optimal['state'] != 'USA']
ARIMA = ARIMA[ARIMA['state'] != 'USA']
RandomForest = RandomForest[RandomForest['state'] != 'USA']
XGBoost = XGBoost[XGBoost['state'] != 'USA']
Combination_AR_1 = Combination_AR_1[Combination_AR_1['state'] != 'USA']
MeanBenchmark = MeanBenchmark[MeanBenchmark['state'] != 'USA']

In [ ]:
# Setting the dataframes to datetime
actuals['Date'] = pd.to_datetime(actuals['Date'])
AR_1['Date'] = pd.to_datetime(AR_1['Date'])
AR_optimal['Date'] = pd.to_datetime(AR_optimal['Date'])
ARIMA['Date'] = pd.to_datetime(ARIMA['Date'])
RandomForest['Date'] = pd.to_datetime(RandomForest['Date'])
XGBoost['Date'] = pd.to_datetime(XGBoost['Date'])
Combination_AR_1['Date'] = pd.to_datetime(Combination_AR_1['Date'])
MeanBenchmark['Date'] = pd.to_datetime(MeanBenchmark['Date'])

In [ ]:
# Setting the dataframes to have the same length
actuals = actuals[actuals['Date'] >= pd.Timestamp('2008-07-01')].reset_index(drop=True)
AR_1 = AR_1[AR_1['Date'] >= pd.Timestamp('2008-07-01')].reset_index(drop=True)
AR_optimal = AR_optimal[AR_optimal['Date'] >= pd.Timestamp('2008-07-01')].reset_index(drop=True)
ARIMA = ARIMA[ARIMA['Date'] >= pd.Timestamp('2008-07-01')].reset_index(drop=True)
RandomForest = RandomForest[RandomForest['Date'] >= pd.Timestamp('2008-07-01')].reset_index(drop=True)
XGBoost = XGBoost[XGBoost['Date'] >= pd.Timestamp('2008-07-01')].reset_index(drop=True)
Combination_AR_1 = Combination_AR_1[Combination_AR_1['Date'] >= pd.Timestamp('2008-07-01')].reset_index(drop=True)
MeanBenchmark = MeanBenchmark[MeanBenchmark['Date'] >= pd.Timestamp('2008-07-01')].reset_index(drop=True)

# MSE over time vs. Actuals

In [ ]:
# List of models used in making MSE over time
models = {
    'XGBoost': XGBoost,
    'Random Forest': RandomForest,
    'AR(Optimal)': AR_optimal,
    'ARIMA': ARIMA,
    'Combination': Combination_AR_1
}

In [ ]:
horizons = ['h1', 'h3', 'h6', 'h12']
mse_entries = []

for model_name, df_model in models.items():
    for horizon in horizons:
        # Combining actuals and predictions into one df
        combined = pd.DataFrame({
            'Date': actuals['Date'],
            'State': actuals['state'],
            'Actuals': actuals[horizon],
            'Predictions': df_model[horizon]
        }).dropna()

        # Calculate MSE
        combined['Squared_Error'] = (combined['Actuals'] - combined['Predictions']) ** 2
        mse_by_date = combined.groupby('Date')['Squared_Error'].mean().reset_index()

        # Calculating cumulative MSE and append to one df
        mse_by_date['Cumulative_Squared_Difference'] = mse_by_date['Squared_Error'].cumsum()
        mse_by_date['Model'] = model_name
        mse_by_date['Horizon'] = horizon
        mse_entries.extend(mse_by_date.to_dict('records'))

cumulative_mse_df = pd.DataFrame(mse_entries)

# MSE over time vs. MeanBenchmark

In [ ]:
# List of model dataframes and their identifiers
models = {
    'XGBoost': XGBoost,
    'Random Forest': RandomForest,
    'AR(Optimal)': AR_optimal,
    'ARIMA': ARIMA,
    'Combination': Combination_AR_1,
    'MeanBenchmark': MeanBenchmark
}

In [ ]:
horizons = ['h1', 'h3', 'h6', 'h12']
benchmark_model_name = 'MeanBenchmark'
entries = []
benchmark_data = {}

# Combining df's
for horizon in horizons:
    combined_benchmark = pd.DataFrame({
        'Date': actuals['Date'],
        'State': actuals['state'],
        'Actuals': actuals[horizon],
        'Benchmark_Predictions': models[benchmark_model_name][horizon]
    }).dropna()

    avg_benchmark = combined_benchmark.groupby('Date').agg({
        'Benchmark_Predictions': 'mean'
    }).reset_index()

    benchmark_data[horizon] = avg_benchmark

# Iterate on each model excluding the benchmark
for model_name, df_model in models.items():
    if model_name != benchmark_model_name:
        for horizon in horizons:
            combined = pd.DataFrame({
                'Date': actuals['Date'],
                'State': actuals['state'],
                'Actuals': actuals[horizon],
                'Predictions': df_model[horizon]
            }).dropna()

            # Calculate MSE
            combined['Squared_Error'] = (combined['Actuals'] - combined['Predictions']) ** 2
            mse_by_date = combined.groupby('Date')['Squared_Error'].mean().reset_index()
            mse_by_date = mse_by_date.join(benchmark_data[horizon].set_index('Date'), on='Date')
            mse_by_date['Squared_Difference'] = mse_by_date['Benchmark_Predictions'] ** 2 - combined.groupby('Date')['Predictions'].mean().reset_index()['Predictions'] ** 2

            # Storing results
            for index, row in mse_by_date.iterrows():
                entries.append({
                    'Model': model_name,
                    'Horizon': horizon,
                    'Date': row['Date'],
                    'Squared_Error': row['Squared_Error'],
                    'Squared_Difference': row['Squared_Difference']
                })

# Making cumulative MSE
result_Mean = pd.DataFrame(entries)
result_Mean.sort_values(by=['Model', 'Horizon', 'Date'], inplace=True)
result_Mean['Cumulative_Squared_Difference'] = result_Mean.groupby(['Model', 'Horizon'])['Squared_Difference'].cumsum()

# Models vs. AR(1)

In [ ]:
# List of model dataframes and their identifiers
models = {
    'AR_1': AR_1,
    'XGBoost': XGBoost,
    'Random Forest': RandomForest,
    'AR(Optimal)': AR_optimal,
    'ARIMA': ARIMA,
    'Combination': Combination_AR_1,
}

In [ ]:
horizons = ['h1', 'h3', 'h6', 'h12']
benchmark_model_name = 'AR_1'
entries = []
benchmark_data = {}

# Combining df's
for horizon in horizons:
    combined_benchmark = pd.DataFrame({
        'Date': actuals['Date'],
        'State': actuals['state'],
        'Actuals': actuals[horizon],
        'Benchmark_Predictions': models[benchmark_model_name][horizon]
    }).dropna()

    avg_benchmark = combined_benchmark.groupby('Date').agg({
        'Benchmark_Predictions': 'mean'
    }).reset_index()

    benchmark_data[horizon] = avg_benchmark

# Iterate on each model excluding the benchmark
for model_name, df_model in models.items():
    if model_name != benchmark_model_name:
        for horizon in horizons:
            combined = pd.DataFrame({
                'Date': actuals['Date'],
                'State': actuals['state'],
                'Actuals': actuals[horizon],
                'Predictions': df_model[horizon]
            }).dropna()

            # Calculate MSE
            combined['Squared_Error'] = (combined['Actuals'] - combined['Predictions']) ** 2
            mse_by_date = combined.groupby('Date')['Squared_Error'].mean().reset_index()
            mse_by_date = mse_by_date.join(benchmark_data[horizon].set_index('Date'), on='Date')
            mse_by_date['Squared_Difference'] = mse_by_date['Benchmark_Predictions'] ** 2 - combined.groupby('Date')['Predictions'].mean().reset_index()['Predictions'] ** 2

            # Storing results
            for index, row in mse_by_date.iterrows():
                entries.append({
                    'Model': model_name,
                    'Horizon': horizon,
                    'Date': row['Date'],
                    'Squared_Error': row['Squared_Error'],
                    'Squared_Difference': row['Squared_Difference']
                })

# Making cumulative MSE
result_AR = pd.DataFrame(entries)
result_AR.sort_values(by=['Model', 'Horizon', 'Date'], inplace=True)
result_AR['Cumulative_Squared_Difference'] = result_AR.groupby(['Model', 'Horizon'])['Squared_Difference'].cumsum()

# Plot

In [ ]:
# Changing units
result_Mean['Cumulative_Squared_Difference'] = result_Mean['Cumulative_Squared_Difference']*100
result_AR['Cumulative_Squared_Difference'] = result_AR['Cumulative_Squared_Difference']*100
cumulative_mse_df['Cumulative_Squared_Difference'] = cumulative_mse_df['Cumulative_Squared_Difference']*100

# Setting font styles
plt.rcParams.update({'font.size': 8, 'font.style': 'normal'})

fig, axes = plt.subplots(nrows=4, ncols=3, figsize=(11, 16), sharex=True)

# Setting horizon, benchmark and models
horizons = ['h1', 'h3', 'h6', 'h12']
y_label = [r'$h_1$', r'$h_3$', r'$h_6$', r'$h_{12}$']
benchmarks = ['Actuals', 'AR(1)', 'Mean']
models = ['XGBoost', 'Random Forest', 'AR(Optimal)', 'ARIMA', 'Combination']

# Collection of dataframes
data_sources = {
    'Actuals': cumulative_mse_df,
    'AR(1)': result_AR,
    'Mean': result_Mean
}

# Plotting the data
for i, horizon in enumerate(horizons):
    for j, benchmark in enumerate(benchmarks):
        ax = axes[i, j]
        df = data_sources[benchmark]
        min_y = float('inf')
        max_y = float('-inf')

        for model in models:
            subset = df[(df['Model'] == model) & (df['Horizon'] == horizon)]
            if not subset.empty:
                ax.plot(subset['Date'], subset['Cumulative_Squared_Difference'], label=model, linewidth=0.9)
                min_y = min(min_y, subset['Cumulative_Squared_Difference'].min())
                max_y = max(max_y, subset['Cumulative_Squared_Difference'].max())

        if i == 0:
            ax.set_title(benchmark, fontsize=12)
            if j == 0:
                ax.set_ylim(-0.05, 0.55)
                ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.2f'))
                ax.yaxis.set_major_locator(ticker.MultipleLocator(0.1))
                ax.tick_params(axis='y', labelsize=9)
                ax.legend(loc='upper left', prop={'size': 8}, frameon=False)

            elif j == 1:
                ax.set_ylim(-0.015, 0.165)
                ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.2f'))
                ax.yaxis.set_major_locator(ticker.MultipleLocator(0.05))
                ax.tick_params(axis='y', labelsize=9)
                ax.legend(loc='upper left', prop={'size': 8}, frameon=False)

            elif j == 2:
                ax.set_ylim(-0.325, 0.025)
                ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.2f'))
                ax.yaxis.set_major_locator(ticker.MultipleLocator(0.1))
                ax.tick_params(axis='y', labelsize=9)
                ax.legend(loc='lower left', prop={'size': 8}, frameon=False)

        elif i == 1:
            if j == 0:
                ax.set_ylim(-0.25, 6.25)
                ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.1f'))
                ax.yaxis.set_major_locator(ticker.MultipleLocator(1))
                ax.tick_params(axis='y', labelsize=9)
                ax.legend(loc='upper left', prop={'size': 8}, frameon=False)

            elif j == 1:
                ax.set_ylim(-0.075, 1.45)
                ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.1f'))
                ax.yaxis.set_major_locator(ticker.MultipleLocator(0.2))
                ax.tick_params(axis='y', labelsize=9)
                ax.legend(loc='upper left', prop={'size': 8}, frameon=False)

            elif j == 2:
                ax.set_ylim(-2.1, 0.1)
                ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.1f'))
                ax.yaxis.set_major_locator(ticker.MultipleLocator(0.4))
                ax.tick_params(axis='y', labelsize=9)
                ax.legend(loc='lower left', prop={'size': 8}, frameon=False)
        elif i == 2:
            if j == 0:
                ax.set_ylim(-1.3, 21.3)
                ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.1f'))
                ax.yaxis.set_major_locator(ticker.MultipleLocator(5))
                ax.tick_params(axis='y', labelsize=9)
                ax.legend(loc='upper left', prop={'size': 8}, frameon=False)
            elif j == 1:
                ax.set_ylim(-0.25, 3.25)
                ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.1f'))
                ax.yaxis.set_major_locator(ticker.MultipleLocator(1))
                ax.tick_params(axis='y', labelsize=9)
                ax.legend(loc='upper left', prop={'size': 8}, frameon=False)
            elif j == 2:
                ax.set_ylim(-5.4, 0.4)
                ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.1f'))
                ax.yaxis.set_major_locator(ticker.MultipleLocator(1))
                ax.tick_params(axis='y', labelsize=9)
                ax.legend(loc='lower left', prop={'size': 8}, frameon=False)
        elif i == 3:
            if j == 0:
                ax.set_ylim(-5, 75)
                ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.1f'))
                ax.yaxis.set_major_locator(ticker.MultipleLocator(10))
                ax.tick_params(axis='y', labelsize=9)
                ax.legend(loc='upper left', prop={'size': 8}, frameon=False)
                ax.set_xlabel('Year', rotation=0, labelpad=10, fontsize=12)
            elif j == 1:
                ax.set_ylim(-3.5, 5)
                ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.1f'))
                ax.yaxis.set_major_locator(ticker.MultipleLocator(1.5))
                ax.tick_params(axis='y', labelsize=9)
                ax.legend(loc='lower left', prop={'size': 8}, frameon=False)
                ax.set_xlabel('Year', rotation=0, labelpad=10, fontsize=12)
            elif j == 2:
                ax.set_ylim(-15, 1)
                ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.1f'))
                ax.yaxis.set_major_locator(ticker.MultipleLocator(2))
                ax.tick_params(axis='y', labelsize=9)
                ax.legend(loc='lower left', prop={'size': 8}, frameon=False)
                ax.set_xlabel('Year', rotation=0, labelpad=10, fontsize=12)

        if j == 0:
            ax.set_ylabel(f'{y_label[i]} CMSE', labelpad=10, fontsize=12)

        # Date formatting
        ax.xaxis.set_major_locator(mdates.YearLocator(4))
        ax.xaxis.set_minor_locator(mdates.YearLocator(1))
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
        plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, ha="center", fontsize=9)

plt.tight_layout()
plt.subplots_adjust(top=0.9, wspace=0.2, hspace=0.1)
plt.show()

In [ ]:
# END